<a href="https://colab.research.google.com/github/edoardorossi1995/Tesi/blob/main/Learning_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pseudocodice 
###Algoritmo di tipo Q-learning, off policy, a task non episodico, che approssima la funzione valore con una rete neurale, con un framework di tipo average reward.

Inizializza la rete neurale per approssimare la funzione Q

Inizializza la media delle ricompense future attese per ogni stato-azione a zero

Ripeti finché non raggiungi la fine del processo:

- S = stato corrente

- Seleziona un'azione A seguendo la politica di esplorazione (ad esempio, la politica ε-greedy)
- Esegui l'azione A e ottieni il prossimo stato S' e la ricompensa R
- Calcola la media delle ricompense future attese per lo stato-azione corrente utilizzando la formula:
- - media_ricompense[S, A] = (1 - α) * media_ricompense[S, A] + α * (R + γ * max(Q(S', a), rete_neurale))
- Calcola il valore target utilizzando la formula:
- - target = media_ricompense[S, A]
- Aggiorna la rete neurale utilizzando il valore target come obiettivo e il vettore di stato-azione come input S = S'

Restituisci la **rete neurale** ottenuta







#Drive Setting


In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB == True:
  from google.colab import drive
  drive.mount('/content/gdrive')
  %cd /content/gdrive/MyDrive/Tesi
  #!ls

import warnings
import pickle
import sys
sys.path.insert(0, '/content/gdrive/MyDrive/Tesi/functions')
sys.path.insert(0, '/content/gdrive/MyDrive/Tesi')

warnings.filterwarnings('ignore')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Tesi


In [2]:
!ls '/content/gdrive/MyDrive/Tesi/functions'

compressor.py  pkl.py  __pycache__


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from keras.models import Sequential
from keras.layers import Dense, Lambda
from keras.activations import exponential, relu
from keras import backend as K

from pkl import store_data, load_data
from functions.compressor import compress

import random
from collections import defaultdict

# Import Anagrafica e CTA

In [4]:
#import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt

CTA30_def = ['/content/gdrive/MyDrive/Tesi/CTA30/DF_clean/clean_CTA_30_def.csv']
df_def = pd.read_csv(CTA30_def[0])

CTA30_18 = ['/content/gdrive/MyDrive/Tesi/CTA30/DF_clean/clean_CTA_30_2018.csv']
df = pd.read_csv(CTA30_18[0])

pca_df = compress(df)

In [5]:
print("Anagrafica shape:",df_def.shape,"\nDataframe shape:", df.shape,"\nPca Dataframe shape:", pca_df.shape)

Anagrafica shape: (76, 2) 
Dataframe shape: (141811, 76) 
Pca Dataframe shape: (141811, 7)


In [6]:
df_def

,pointColName,pointName
0,AUTOMAN_SERR_CMD_00,Automatico Manuale Serrande Comando
1,AUTOMAN_VLV_CMD_00,Automatico Manuale Valvole Comando
2,AUTOMAN_M_CMD_00,Automatico Manuale Mandata Comando
3,AUTOMAN_R_CMD_00,Automatico Manuale Ripresa Comando
4,LOCREM_CMD_00,Selettore Locale Remoto Comando
...,...,...
71,VOLT_R_00,Tensione Ripresa
72,T_M_SP_00,Temperatura Aria Mandata Setpoint
73,T_R_SP_00,Temperatura Aria Ritorno Setpoint
74,T_AMB_04,Temperatura Aria Ambiente Media


# Rete Neurale

Log-sum-exp neural networks and posynomial
models for convex and log-log-convex data 

(Calafiore, Gaubert, Possieri)

In [39]:
n = load_data('*.pkl')

# Funzione di attivazione logatirmica (ultimo strato)
def log_act(x):
  return K.log(x)

# Architettura rete neurale
input_dim = n['n_components']+1
inner_dim_1 = 32
output_dim = 1

print("input dim: ", input_dim)
print("inner layer dim: ", inner_dim_1)
print("output dim: ", output_dim)

# Costruzione strati
rete_neurale = Sequential()
rete_neurale.add(Dense(units = inner_dim_1, activation = exponential, input_dim = input_dim))
rete_neurale.add(Dense(units = output_dim, activation=log_act)) 

rete_neurale.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])

input dim:  8
inner layer dim:  32
output dim:  1


# Q-Learning Agent

In [40]:
# Dimensione problema e inizializzazione stato iniziale
[S, feat] = df.shape
current_state = np.random.randint(0,S-1)

# Inizializza la media delle ricompense future attese per ogni stato-azione a zero
q = defaultdict(float)
rewards = []

# Parametri di apprendimento
# alpha = 1
# gamma = 1

total_episodes = 10
current_episode = 0

# Stato corrente
s = current_state

# Ripeti finché non raggiungi la fine del processo
for current_episode in range(total_episodes):
    
    print("Episodio ",current_episode,"/",total_episodes)

    # Stato successivo e ottieni l'azione A e la ricompensa R
    s_prime = s+1

    a = df.loc[s,'F_M_FDBK_00']-df.loc[s_prime,'F_M_FDBK_00']
    r = -df.loc[s,'PW_ATT_M_00']
    rewards.append(r)

    # Calcola la media delle ricompense future attese per lo stato-azione corrente
    # media_ricompense[(s, a)] = (1 - alpha) * media_ricompense[(s, a)] + alpha * (r + gamma * np.max(rete_neurale.predict(s_prime)))
    q[(s,a)] = r - np.mean(rewards)

    # Calcola il valore target
    target = q[(s,a)]
    print("target: ", target, "\nr: ",r,"\nreward medio: ",np.mean(rewards))

    # Comprimo lo stato e genero l'input della rete neurale
    pca_s = pca_df.loc[s,:]

    nn_input = np.concatenate((pca_s.values, np.array([a])))
    nn_input = np.reshape(nn_input, (1,input_dim))
    target = np.array([target])

    # Aggiorna la rete neurale
    rete_neurale.fit(nn_input, target, epochs=1, verbose=0)

    # Aggiorna lo stato corrente
    s = s_prime

    print("\n")

# Restituisci la rete neurale ottenuta
#return rete_neurale

Episodio  0 / 10
target:  -0.0 
r:  -0.0 
reward medio:  0.0


Episodio  1 / 10
target:  -0.0 
r:  -0.0 
reward medio:  0.0


Episodio  2 / 10
target:  -0.0 
r:  -0.0 
reward medio:  0.0


Episodio  3 / 10
target:  -0.0 
r:  -0.0 
reward medio:  0.0


Episodio  4 / 10
target:  -0.0 
r:  -0.0 
reward medio:  0.0


Episodio  5 / 10
target:  -0.0 
r:  -0.0 
reward medio:  0.0


Episodio  6 / 10
target:  -0.0 
r:  -0.0 
reward medio:  0.0


Episodio  7 / 10
target:  -0.0 
r:  -0.0 
reward medio:  0.0


Episodio  8 / 10
target:  -0.0 
r:  -0.0 
reward medio:  0.0


Episodio  9 / 10
target:  -0.0 
r:  -0.0 
reward medio:  0.0


